# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Import API key
from api_keys import geoapify_key

In [53]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ullapool,57.8987,-5.1604,10.19,74,100,2.92,GB,1681606886
1,1,ushuaia,-54.8000,-68.3000,6.81,70,75,2.06,AR,1681606827
2,2,hilo,19.7297,-155.0900,27.36,83,35,2.24,US,1681606898
3,3,camana,-16.6228,-72.7111,22.48,83,88,2.86,PE,1681606898
4,4,naze,28.3667,129.4833,20.51,60,0,3.69,JP,1681606899
...,...,...,...,...,...,...,...,...,...,...
559,559,santa cruz del sur,20.7147,-77.9956,26.45,73,100,3.41,CU,1681607220
560,560,luanda,-8.8368,13.2343,26.13,88,100,4.83,AO,1681607259
561,561,valley city,46.9233,-98.0032,0.29,100,100,7.72,US,1681607246
562,562,miramar,25.9873,-80.2323,27.22,82,0,4.12,US,1681607259


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [54]:
%%capture --no-display
#Configure the map plot and display the map
each_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 670,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

each_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [55]:
fig = each_city_map
plt.savefig("../output_data/VacFig1.png")

<Figure size 640x480 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_vacay_df = pd.DataFrame(city_data_df.loc[(city_data_df["Max Temp"] >=15) & (city_data_df["Max Temp"] <=25) 
                                 & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0), :])
perfect_vacay_df.reset_index(drop=True, inplace=True)
                                
# Drop any rows with null values
perfect_vacay_df.dropna(inplace=True)

# Display sample data
perfect_vacay_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4,naze,28.3667,129.4833,20.51,60,0,3.69,JP,1681606899
1,40,diego de almagro,-26.3667,-70.0500,15.99,56,0,1.00,CL,1681606915
2,44,cape town,-33.9258,18.4232,17.43,68,0,6.71,ZA,1681606815
3,52,ponta do sol,32.6667,-17.1000,18.89,67,0,2.02,PT,1681606920
4,57,cabo san lucas,22.8909,-109.9124,24.78,62,0,1.34,MX,1681606803


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df = pd.DataFrame(perfect_vacay_df, columns = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"])

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,naze,JP,28.3667,129.4833,60,NaN
1,diego de almagro,CL,-26.3667,-70.0500,56,NaN
2,cape town,ZA,-33.9258,18.4232,68,NaN
3,ponta do sol,PT,32.6667,-17.1000,67,NaN
4,cabo san lucas,MX,22.8909,-109.9124,62,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
distance = 10000
categories = "accomodation.hotel"
limit = 1

params = {
   "distance": distance,
   "categories": categories,
    "limit": limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"cirlce:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v1/geocode/search?REQUEST_PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
#name_address

Starting hotel search
naze - nearest hotel: No hotel found
diego de almagro - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
ponta do sol - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
redding - nearest hotel: No hotel found
sao filipe - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
dunedin - nearest hotel: No hotel found
san ramon - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
prescott - nearest hotel: No hotel found
silver city - nearest hotel: No hotel found
hun - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
salinas - nearest hotel: No hotel found
ahome - nearest hotel: No hotel found
bushehr - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
kalabo - nearest hotel: No hotel found
saint george - nearest hotel: No hotel found
linxia - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
0,naze,JP,28.3667,129.4833,60,No hotel found
1,diego de almagro,CL,-26.3667,-70.0500,56,No hotel found
2,cape town,ZA,-33.9258,18.4232,68,No hotel found
3,ponta do sol,PT,32.6667,-17.1000,67,No hotel found
4,cabo san lucas,MX,22.8909,-109.9124,62,No hotel found
5,redding,US,40.5865,-122.3917,37,No hotel found
6,sao filipe,CV,14.8961,-24.4956,74,No hotel found
7,geraldton,AU,-28.7667,114.6000,63,No hotel found
8,dunedin,NZ,-45.8742,170.5036,51,No hotel found
9,san ramon,US,37.7799,-121.9780,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
vac_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 670,
    frame_height = 500, 
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
vac_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)